In [33]:
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader,PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import ChatGooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langchain_groq import ChatGroq

In [34]:
load_dotenv()

True

In [35]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [36]:
llm = ChatGroq(
    model='llama-3.3-70b-versatile',
    temperature=0.3
)

In [37]:
from langchain_community.document_loaders import PyPDFLoader
def get_data(file_path):
    loader = PyPDFLoader(file_path)
    data = loader.load()
    return data 

In [38]:
data = get_data("LingeswaranMithurshanResume.pdf")

In [39]:
def get_chunks(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                   chunk_overlap = 200)
    docs = splitter.split_documents(data)
    return docs

In [40]:
docs = get_chunks(data)

In [41]:
def get_embedding_vecter_index(docs):
    vecter_store = FAISS.from_documents(documents=docs,embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))
    return vecter_store

In [42]:
vector_db = get_embedding_vecter_index(docs)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [44]:
def get_rag_chain(vector_db):
    system_prompt = (
    """
    You are a Resume Analyzer Assistant. Your job is to evaluate resumes and provide clear, structured, and insightful feedback on skills, occupation, and overall structure.

    ### Your tasks:
    - **Identify strengths**
    - **Suggest improvements**
    - **Ensure alignment with industry standards**
    - **Provide concise and actionable recommendations**

    ### Guidelines:
    1. If the user asks about something not available in the resume, generate a reasonable answer based on industry standards.
    2. If the user's question is not relevant to resume analysis, respond with:
    **"I don't know. This question is not relevant for me. I am here to help you analyze and improve your resume."**
    3. **Use bullet points or numbered lists** for structured responses.
    4. **Keep answers short and to the point**.
    5. **Ensure markdown-friendly formatting** for better readability.

    ### Example Format:
    **Strengths:**
    - ✅ Clear experience in software development  
    - ✅ Strong skills in Python and SQL  

    **Areas for Improvement:**
    1. Improve resume structure by adding a summary section  
    2. Include specific project details with impact metrics  
    3. Highlight technical skills using a separate section  

    {context}
    """
)
    prompt = ChatPromptTemplate.from_messages([
        ("system",system_prompt),
        ("human","{input}"),
    ])
    retriever = vector_db.as_retriever()
    qa_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)
    rag_chain= create_retrieval_chain(retriever,qa_chain)
    return rag_chain
    

In [45]:
rag_chain = get_rag_chain(vector_db)

In [46]:
respnse = rag_chain.invoke({"input":"what is my career path?"})
respnse['answer']

'**Career Path:**\nBased on your resume, your career path appears to be in the field of **Data Science*, specifically as a **Data Scientist*. \n\n**Key Indicators:**\n- Your job title is listed as "Data Scientist".\n- Your skills section highlights skills in machine learning, deep learning, computer vision, NLP, and data visualization.\n- Your experience and projects demonstrate a strong focus on data analytics, machine learning, and data science.\n\n**Potential Career Progression:**\n1. **Junior Data Scientist**: You are currently in this role, focusing on building a strong foundation in data science and machine learning.\n2. **Senior Data Scientist**: With experience, you can move into a senior role, leading projects and mentoring junior data scientists.\n3. **Data Science Manager**: You can progress into a management role, overseeing data science teams and driving business strategy.\n4. **Director of Data Science**: In this role, you will lead the data science function, driving inno

In [47]:

respnse = rag_chain.invoke({"input":"i would like to shift my career path into software engineer,is possible what can i do can you provide the answer like points form?"})
respnse['answer']

"**Career Transition to Software Engineer:**\n### Key Considerations:\n* Your current background is in Data Science, which shares some similarities with Software Engineering, making a transition possible.\n* You'll need to acquire new skills and build on your existing knowledge to become a competitive candidate.\n\n### Actionable Steps:\n1. **Identify transferable skills**: Leverage your experience in programming languages like Python, data structures, and algorithms to build a foundation in software engineering.\n2. **Learn new programming languages**: Familiarize yourself with languages commonly used in software engineering, such as Java, C++, or JavaScript.\n3. **Develop software engineering skills**:\n\t* Data structures and algorithms\n\t* Object-Oriented Programming (OOP) concepts\n\t* Design patterns\n\t* Testing and debugging techniques\n4. **Get familiar with software engineering frameworks and tools**:\n\t* Version control systems like Git\n\t* Agile development methodologies

In [48]:

respnse = rag_chain.invoke({"input":"in few days im suffering from heavy fever what can i do ?"})
respnse['answer']

"I don't know. This question is not relevant for me. I am here to help you analyze and improve your resume. \n\nIf you'd like to discuss your resume or any related topics, I'd be happy to assist you. Please feel free to ask any questions or share your concerns about your resume."

In [49]:
respnse = rag_chain.invoke({"input":"thank you for you answer its usefull for me thank you one again"})
respnse['answer']

"You're welcome. I'm glad I could help. If you have any more questions or need further assistance with your resume, feel free to ask. \n\n**Next Steps:**\n1. Review your resume and implement the suggested improvements.\n2. Tailor your resume to your target job or industry.\n3. Practice answering common interview questions related to your field.\n\nIf you need help with anything else, just let me know."